In [ ]:
!pip install keras_vggface

In [ ]:
!pip install keras_applications

In [ ]:
!pip install mtcnn

In [ ]:
!pip install Pillow

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

In [1]:
from PIL import Image

In [2]:
import tensorflow as tf
import keras
from keras_vggface.vggface import VGGFace
import cv2

2023-08-28 10:40:46.892653: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-28 10:40:47.825517: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
vggface = VGGFace(model='vgg16')

2023-08-28 10:40:50.509783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-28 10:40:50.669717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-08-28 10:40:50.670159: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
train_dataset = keras.utils.image_dataset_from_directory('datasets_VGG',
                                                         shuffle=True,
                                                         batch_size=16,
                                                         image_size=(224,224))

Found 1792 files belonging to 4 classes.


In [5]:
data_aug = keras.Sequential([
    tf.keras.layers.RandomFlip('horizontal'),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomBrightness(0.1),
    tf.keras.layers.RandomContrast(0.1)
])

In [6]:
vggface_resnet_base = VGGFace(model='resnet50')

In [7]:
print('Inputs: %s' % vggface_resnet_base.inputs)
print('Outputs: %s' % vggface_resnet_base.outputs)

Inputs: [<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'input_2')>]
Outputs: [<KerasTensor: shape=(None, 8631) dtype=float32 (created by layer 'classifier')>]


In [8]:
from keras.layers import Flatten, Dense, Input
from keras_vggface.vggface import VGGFace

In [9]:
number_class = 4

In [10]:
vggface_resnet_base.trainable = False
last_layer = vggface_resnet_base.get_layer('avg_pool').output

inputs = tf.keras.Input(shape=(224,224,3))

x = data_aug(inputs)
x = vggface_resnet_base(x)
x = Flatten(name='flatten')(x)

out = Dense(4, name='classifier')(x)
custom_model = keras.Model(inputs, out)

In [11]:
base_lr = 0.0005

custom_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=base_lr),
                     loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                     metrics=['accuracy'])

In [12]:
custom_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 sequential (Sequential)     (None, 224, 224, 3)       0         
                                                                 
 vggface_resnet50 (Functiona  (None, 8631)             41246071  
 l)                                                              
                                                                 
 flatten (Flatten)           (None, 8631)              0         
                                                                 
 classifier (Dense)          (None, 4)                 34528     
                                                                 
Total params: 41,280,599
Trainable params: 34,528
Non-trainable params: 41,246,071
____________________________________________

In [13]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                  patience=5)

In [14]:
his = custom_model.fit(train_dataset, epochs=300, callbacks=[early_stopping])

Epoch 1/300


2023-08-28 10:40:54.048407: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [1792]
	 [[{{node Placeholder/_4}}]]
2023-08-28 10:40:54.048673: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [1792]
	 [[{{node Placeholder/_4}}]]
2023-08-28 10:40:57.573904: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-08-28 10:40:58.833710: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-08-28 10

  3/112 [..............................] - ETA: 4s - loss: 1.3861 - accuracy: 0.2292   

2023-08-28 10:40:59.680838: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


112/112 [==============================] - 10s 39ms/step - loss: 1.3713 - accuracy: 0.4330
Epoch 2/300
 81/112 [====================>.........] - ETA: 1s - loss: 1.3499 - accuracy: 0.4035

KeyboardInterrupt: 

In [ ]:
prob_model = keras.Sequential([custom_model,
                               tf.keras.layers.Softmax()
                               ])

In [ ]:
sample_set = []
for i in range(1,31) :
    sample = cv2.imread(f'test/000{i}.png')
    sample_set.append(sample)


In [ ]:
sample_set[24].shape

(132, 130, 3)

In [ ]:
def sample_process(sample) :
    sample_reshape = cv2.resize(sample,dsize=(224,224))
    sample = tf.convert_to_tensor(sample_reshape, dtype=tf.float32)
    sample = tf.expand_dims(sample, 0)
    return sample

In [ ]:
sample_set_processed = []

In [ ]:
for sample in sample_set :
    sample_e = sample_process(sample)
    sample_set_processed.append(sample_e)

In [ ]:
sample_set_processed[0].shape

TensorShape([1, 224, 224, 3])

In [ ]:
name = ['bus','elf','petch','unknown']

In [ ]:
num = 0
for sample in sample_set_processed :
    num+=1
    predictions = prob_model.predict(sample)
    print(predictions[0])
    print(f'num : {num}\tclass : {name[predictions[0].argmax()]}\tconf : {predictions[0][predictions[0].argmax()] * 100}%')

1/1 [==============================] - 0s 16ms/step
[0.06329039 0.75122833 0.04254247 0.14293869]
num : 1	class : elf	conf : 75.12283325195312%
1/1 [==============================] - 0s 18ms/step
[0.10758764 0.19758601 0.59756213 0.09726424]
num : 2	class : petch	conf : 59.756213426589966%
1/1 [==============================] - 0s 18ms/step
[0.06460182 0.68680227 0.06177691 0.18681897]
num : 3	class : elf	conf : 68.68022680282593%
1/1 [==============================] - 0s 16ms/step
[0.80379087 0.05166602 0.05190339 0.09263978]
num : 4	class : bus	conf : 80.37908673286438%
1/1 [==============================] - 0s 18ms/step
[0.26163453 0.16609798 0.10518102 0.4670865 ]
num : 5	class : unknown	conf : 46.70864939689636%
1/1 [==============================] - 0s 15ms/step
[0.310454   0.2051407  0.1673983  0.31700698]
num : 6	class : unknown	conf : 31.700697541236877%
1/1 [==============================] - 0s 15ms/step
[0.40709114 0.0982035  0.10841154 0.3862939 ]
num : 7	class : bus	conf :

In [ ]:
#prob_model.save("third_model_face.tf")